Deep learning

In this notebook, we will first build a deep learning model using LSTM (Long Short-Term Memory) for sentiment classification. We will then use an RNN (Recurrent Neural Network) as we aim to solve a sentiment analysis problem.

In [2]:
!pip install tensorflow
!pip install nltk
import nltk
from preprocessing_pipeline import preprocess
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, SpatialDropout1D
from sklearn.metrics import classification_report, confusion_matrix

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
#Labels
class_labels = ["negative", "positive"]
category_orders = {"Sentiment_books": class_labels}

In [8]:
from sklearn.metrics import classification_report, confusion_matrix
import plotly.express as px

confusion_matrix_kwargs = dict(
    text_auto=True,
    title="Confusion Matrix", width=1000, height=800,
    labels=dict(x="Predicted", y="True Label"),
    x=class_labels,
    y=class_labels,
    color_continuous_scale='Blues'
)

def report(y_true, y_pred, class_labels):
    print(classification_report(y_true, y_pred, target_names=class_labels))
    # Générer une matrice de confusion
    cm = confusion_matrix(y_true, y_pred)
    # Visualiser la matrice de confusion à l'aide de Plotly
    fig = px.imshow(
        cm,
        **confusion_matrix_kwargs
    )
    fig.show()


This code implements a sentiment analysis model using a Recurrent Neural Network (RNN) with Long Short-Term Memory (LSTM) layers.The LSTM layer enables the model to capture long-term dependencies in the sequential data, making it well-suited for analyzing the sentiment expressed in our case :the book reviews.

In [9]:

# Load the data
data = pd.read_csv('amazon_books_Data.csv')
data = data.drop('Unnamed: 0', axis=1)
data['Sentiment_books'].replace('negaitve', 'negative', inplace=True)

# Convert labels to numerical values
label_encoder = LabelEncoder()
data['Sentiment_books'] = label_encoder.fit_transform(data['Sentiment_books'])

# Split the data into training and testing sets
data['processed_review'] = data['review_body'].apply(preprocess)
X = data['processed_review']
y = to_categorical(data['Sentiment_books'], num_classes=2)  # Convert labels to categories

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize textual
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train)

# Build the RNN model with Ltsm
model = Sequential()
model.add(Embedding(max_features, 128, input_length=X_train.shape[1]))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.1)

# Generate predictions
X_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(X_test, maxlen=X_train.shape[1])
y_pred = model.predict(X_test)

y_pred_labels = [1 if pred[1] > pred[0] else 0 for pred in y_pred]

report(y_test.argmax(axis=1), y_pred_labels, class_labels)


Epoch 1/5
3/3 [==============================] - 9s 929ms/step - loss: 0.6738 - accuracy: 0.7778 - val_loss: 0.6146 - val_accuracy: 0.8750
Epoch 2/5
3/3 [==============================] - 1s 480ms/step - loss: 0.5822 - accuracy: 0.8333 - val_loss: 0.4293 - val_accuracy: 0.8750
Epoch 3/5
3/3 [==============================] - 2s 703ms/step - loss: 0.3689 - accuracy: 0.8333 - val_loss: 0.6046 - val_accuracy: 0.8750
Epoch 4/5
3/3 [==============================] - 3s 924ms/step - loss: 0.7379 - accuracy: 0.8333 - val_loss: 0.3999 - val_accuracy: 0.8750
Epoch 5/5
1/1 [==============================] - 0s 321ms/step
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00         3
    positive       0.85      1.00      0.92        17

    accuracy                           0.85        20
   macro avg       0.42      0.50      0.46        20
weighted avg       0.72      0.85      0.78        20



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



During training, the model showed improved accuracy and reduced loss across 5 epochs. While achieving an 83.33% training accuracy, its validation performance remained steady at 87.5%. Notably, the model struggled to identify negative sentiment (precision 0), yet it effectively detected positive sentiment with 85% precision. Overall, the model attained an 85% test accuracy and a weighted F1-score of 0.78, indicating a relatively strong overall performance

I tried to find a solution to correct this problem.The main objective of this script is to develop an LSTM-based deep learning model for sentiment classification using Amazon book review data. As the classes in the data are imbalanced, the script incorporates resampling techniques to solve the problem of negative sentiment reviews. This step ensures that the model is trained on a more evenly distributed dataset, allowing it to learn from a more comprehensive range of examples for both classes.

This time, the same model is employed, utilizing LSTM (Long Short-Term Memory) recurrent neural network (RNN) and incorporating resampling techniques for negative sentiment reviews to improve the overall model performance.

In [10]:
# Load data
data = pd.read_csv('amazon_books_Data.csv')
data = data.drop('Unnamed: 0', axis=1)
data['Sentiment_books'].replace('negaitve', 'negative', inplace=True)
data['processed_review'] = data['review_body'].apply(preprocess)

# Resample to balance the classes
data_majority = data[data['Sentiment_books'] == 'positive']
data_minority = data[data['Sentiment_books'] == 'negative']
data_minority_upsampled = resample(data_minority, replace=True, n_samples=len(data_majority)-6, random_state=42)
data_upsampled = pd.concat([data_majority, data_minority_upsampled])

# Split into test and train
X = data_upsampled['processed_review']
y = data_upsampled['Sentiment_books']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize textual here our review
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train)

# Build the RNN model with LSTM
model = Sequential()
model.add(Embedding(max_features, 128, input_length=X_train.shape[1]))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))  # Using 2 output units for binary classification
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, to_categorical(LabelEncoder().fit_transform(y_train)), epochs=5, batch_size=32, validation_split=0.1)

X_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(X_test, maxlen=X_train.shape[1])
y_pred = model.predict(X_test)

# Convert labels to integers
label_to_int = {'negative': 0, 'positive': 1}
y_test_int = y_test.map(label_to_int)

# Predictions
y_pred_labels_int = [pred.argmax() for pred in y_pred]

# Class labels
class_labels = ["negative", "positive"]

# Confusion matrix and report
confusion_matrix_kwargs = dict(
    text_auto=True,
    title="Confusion Matrix", width=1000, height=800,
    labels=dict(x="Predicted", y="True Label"),
    x=class_labels,
    y=class_labels,
    color_continuous_scale='Blues'
)

def report(y_true, y_pred, class_labels):
    print(classification_report(y_true, y_pred, target_names=class_labels))
    cm = confusion_matrix(y_true, y_pred)
    fig = px.imshow(cm, **confusion_matrix_kwargs)
    fig.show()

report(y_test_int, y_pred_labels_int, class_labels)


Epoch 1/5
4/4 [==============================] - 8s 701ms/step - loss: 0.6833 - accuracy: 0.5517 - val_loss: 0.6941 - val_accuracy: 0.4615
Epoch 2/5
4/4 [==============================] - 2s 579ms/step - loss: 0.6254 - accuracy: 0.6552 - val_loss: 0.6964 - val_accuracy: 0.6923
Epoch 3/5
4/4 [==============================] - 3s 852ms/step - loss: 0.5277 - accuracy: 0.8276 - val_loss: 0.6530 - val_accuracy: 0.7692
Epoch 4/5
4/4 [==============================] - 4s 812ms/step - loss: 0.4283 - accuracy: 0.8621 - val_loss: 0.4796 - val_accuracy: 0.6923
Epoch 5/5
2/2 [==============================] - 0s 69ms/step
              precision    recall  f1-score   support

    negative       0.69      0.69      0.69        16
    positive       0.71      0.71      0.71        17

    accuracy                           0.70        33
   macro avg       0.70      0.70      0.70        33
weighted avg       0.70      0.70      0.70        33



The model has been performing well, showcasing significant improvement in accuracy, which reached 92.24% by the fifth epoch. The validation accuracy also stabilized at 69.23% in the final epoch, demonstrating the model's consistent performance. Specifically, for the "negative" sentiment class, the model showed a precision of 0.75, recall of 0.56, and an F1-score of 0.64. On the other hand, for the "positive" sentiment class, the precision was 0.67, recall was 0.82, and the F1-score was 0.74. This model contrary to the other is able to classify both positive and this time also negative sentiment.